In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew
from scipy.stats import boxcox_normmax
from scipy.special import boxcox1p
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.preprocessing import RobustScaler
from category_encoders.target_encoder import TargetEncoder
from category_encoders.backward_difference import BackwardDifferenceEncoder
from category_encoders.ordinal import OrdinalEncoder
from random import randint

In [ ]:
numeric_columns_impute_loader = ['LotFrontage','GarageArea','GarageCars','BsmtHalfBath','BsmtFullBath','TotalBsmtSF','BsmtUnfSF','BsmtFinSF1','BsmtFinSF2','MasVnrArea','GarageYrBlt',
                          'LotArea', 'OverallQual','OverallCond','YearBuilt','YearRemodAdd','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','FullBath',
                         'HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','WoodDeckSF',
                         'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold',
                         'YrSold']
class DataLoader():
    def __init__(self,train_path,test_path):
        train = pd.read_csv(train_path)
        test = pd.read_csv(test_path)
        train = self.__remove_outliers(train)
        self.X_train = train.drop(['Id','SalePrice'], axis = 1)
        self.X_train['MSSubClass'] = self.X_train['MSSubClass'].astype(str)
        self.Y_train = train['SalePrice']
        self.X_test = test.drop(['Id'], axis = 1)
        self.X_test['MSSubClass'] = self.X_test['MSSubClass'].astype(str)
        self.test_Id = test['Id']
        self.X_train, self.X_test = self.impute_numeric(self.X_train, self.X_test,numeric_columns_impute_loader)
        self.X_train = self.feature_engineering(self.X_train)
        self.X_test = self.feature_engineering(self.X_test)
        self.X_train, self.X_test = self.remove_skewness(self.X_train, self.X_test)
        #self.X_train, self.X_test = self.add_polynomials(self.X_train, self.X_test)
    def getX_train(self):
        return self.X_train
    def getX_test(self):
        return self.X_test
    def getY_train(self):
        return self.Y_train
    def get_Test_id(self):
        return self.test_Id
    def __remove_outliers(self,train):
        train = train[train.GrLivArea < 4500]
        train.reset_index(drop=True, inplace=True)
        outliers = [30, 88, 462, 631, 1322]
        train = train.drop(train.index[outliers])
        train.reset_index(drop=True, inplace=True)
        train = train[train.GarageArea < 1200]
        train.reset_index(drop=True, inplace=True)
        return train
    def remove_skewness(self,x_train,x_test):
        x = pd.concat([x_train,x_test])
        numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        numerics = []
        for i in x.columns:
            if x[i].dtype in numeric_dtypes:
                numerics.append(i)
        skew_features = x[numerics].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
        high_skew = skew_features[skew_features > 0.5]
        skew_index = high_skew.index
        for i in skew_index:
            x[i] = boxcox1p(x[i], boxcox_normmax(x[i].dropna() + 1))
        x_train_res = x.iloc[:x_train.shape[0]]
        x_test_res = x.iloc[x_train.shape[0]:]
        return (x_train_res,x_test_res)
    def add_polynomials(self,x_train,x_test):
        x = pd.concat([x_train,x_test])
        numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        numerics = []
        for i in x.columns:
            if x[i].dtype in numeric_dtypes:
                x[i+'square'] = x[i]*x[i]
        x_train_res = x.iloc[:x_train.shape[0]]
        x_test_res = x.iloc[x_train.shape[0]:]
        return (x_train_res,x_test_res)
    def impute_numeric(self,x_train,x_test,cols):
        for i in cols:
            agg = x_train[i].median()
            x_train[i].fillna(agg, inplace=True)
            x_test[i].fillna(agg, inplace=True)
        return x_train,x_test
            
    def feature_engineering(self,features):
        features['YrBltAndRemod']=features['YearBuilt']+features['YearRemodAdd']
        features['TotalSF']=features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']
        features['Total_sqr_footage'] = (features['BsmtFinSF1'] + features['BsmtFinSF2'] +
                                 features['1stFlrSF'] + features['2ndFlrSF'])
        features['Total_Bathrooms'] = (features['FullBath'] + (0.5 * features['HalfBath']) +
                               features['BsmtFullBath'] + (0.5 * features['BsmtHalfBath']))
        features['Total_porch_sf'] = (features['OpenPorchSF'] + features['3SsnPorch'] +
                              features['EnclosedPorch'] + features['ScreenPorch'] +
                              features['WoodDeckSF'])
        features['haspool'] = features['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
        features['has2ndfloor'] = features['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
        features['hasgarage'] = features['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
        features['hasbsmt'] = features['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
        features['hasfireplace'] = features['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)
        return features
        
        

In [ ]:
class ColumnDrop(BaseEstimator,TransformerMixin):
    def __init__(self,columns):
        self.columns = columns
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.drop(self.columns,axis = 1)
class FeatureNormalizer(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        numerics = []
        for i in X.columns:
            if X[i].dtype in numeric_dtypes:
                numerics.append(i)
        skew_features = X[numerics].apply(lambda x: skew(x)).sort_values(ascending=False)
        high_skew = skew_features[skew_features > 0.5]
        self.skew_index = high_skew.index
        return self
    def transform(self, X):
        for i in self.skew_index:
            X[i] = boxcox1p(X[i], boxcox_normmax(X[i].dropna() + 1))
        return X
class ToDataframeConverter(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return pd.DataFrame(X)      
class OverfitDrop(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X

In [ ]:
class TargetNormalizedRegressor(BaseEstimator, RegressorMixin):
    def __init__(self,regressor):
        self.regressor = regressor
    def fit(self, X_train, Y_train):
        return self.regressor.fit(X_train,np.log1p(Y_train))
    def predict(self, X):
        return np.expm1(self.regressor.predict(X))
class BlendingRegressor(BaseEstimator, RegressorMixin):
    def __init__(self,regressors_and_weights):
        self.regressors_and_weights = regressors_and_weights
    def fit(self, X_train, Y_train):
        for regressor, weight in self.regressors_and_weights:
            regressor.fit(X_train,Y_train)
        return self
    def predict(self, X):
        y_pred = 0
        for regressor, weight in self.regressors_and_weights:
            y_pred += weight*regressor.predict(X)
        return y_pred  

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.impute import SimpleImputer
class DfColumnTransformer(ColumnTransformer):
    def transform(self, X):
        return pd.DataFrame(ColumnTransformer.transform(self,X),columns = self.get_feature_names())
    def fit_transform(self, X, y=None):
        return pd.DataFrame(ColumnTransformer.fit_transform(self, X, y), columns = self.get_feature_names())
    def get_feature_names(self):
        feature_names = []
        for name, trans, _, _ in self._iter(fitted=True):
            if trans == 'drop':
                continue
            feature_names.extend([f for f in
                                  trans.get_feature_names()])
        return feature_names
    
class PipelineWithFeatureNames(Pipeline):
    def __init__(self, steps, memory=None, verbose=False, feature_names = None):
        Pipeline.__init__(self, steps, memory,verbose)
        self.feature_names = feature_names
    def get_feature_names(self):
        last_step = self._final_estimator
        if isinstance(last_step, OneHotEncoder) and self.feature_names:
             return last_step.get_feature_names(input_features = self.feature_names)
        else:
             return self.feature_names    
class SimpleImputerWithFeatureNames(SimpleImputer):
    def __init__(self, missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True, add_indicator=False, feature_names = None):
        SimpleImputer.__init__(self, missing_values,strategy,fill_value,verbose,copy,add_indicator)
        self.feature_names = feature_names
    def get_feature_names(self):
        return self.feature_names    
    
categorial_features = ['LotShape','LandContour','LandSlope',
            'LotConfig','Neighborhood',
            'BldgType','HouseStyle',
            'RoofStyle','RoofMatl',
            'ExterQual','ExterCond','Foundation','Heating','HeatingQC',
            'CentralAir',
            'PavedDrive','SaleCondition']

categorial_features_target = []



categorial_features_ignore_unknown = ['MSSubClass','Condition1','Condition2']

categorial_features_with_missing_values_const = ['BsmtQual','Alley','BsmtCond','BsmtExposure','BsmtFinType1',
                                           'BsmtFinType2','FireplaceQu','GarageType','GarageQual','GarageFinish',
                                           'GarageCond','Fence','MiscFeature']
categorial_features_with_missing_values_const_target = []

categorial_features_with_missing_values_impute = ['SaleType','Functional','KitchenQual','Exterior1st','Exterior2nd','MSZoning','MasVnrType','Electrical']

categorial_features_with_missing_values_impute_target = []

numeric_columns_impute = ['LotFrontage','GarageArea','GarageCars','BsmtHalfBath','BsmtFullBath','TotalBsmtSF','BsmtUnfSF','BsmtFinSF1','BsmtFinSF2','MasVnrArea','GarageYrBlt',
                          'LotArea', 'OverallQual','OverallCond','YearBuilt','YearRemodAdd','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','FullBath',
                         'HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','WoodDeckSF',
                         'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold',
                         'YrSold','YrBltAndRemod','TotalSF', 'Total_sqr_footage','Total_Bathrooms','Total_porch_sf',
                         'haspool','has2ndfloor','hasgarage','hasbsmt','hasfireplace']
categorical_with_missing_values_pipeline_const = lambda: PipelineWithFeatureNames(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value="Nan")),
    ('onehot', OneHotEncoder(sparse=False,dtype=int,categories = 'auto'))], feature_names = categorial_features_with_missing_values_const)
categorical_with_missing_values_pipeline_const_target = lambda: PipelineWithFeatureNames(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value="Nan")),
    ('onehot', TargetEncoder())], feature_names = categorial_features_with_missing_values_const_target)
#for now let it be most frequent- median is not supported for numeric but seems same as frequent, but further maybe use advanced imputer that uses ml
categorical_with_missing_values_pipeline_impute = lambda: PipelineWithFeatureNames(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse=False,dtype=int,categories = 'auto'))], feature_names = categorial_features_with_missing_values_impute)

categorical_with_missing_values_pipeline_impute_target = lambda: PipelineWithFeatureNames(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse=False,dtype=int,categories = 'auto'))], feature_names = categorial_features_with_missing_values_impute_target)


#categorical_pipeline = lambda: PipelineWithFeatureNames(steps=[
 #   ('onehot', OneHotEncoder(sparse=False,dtype=int,categories = 'auto'))], feature_names = categorial_features)
categorical_pipeline = lambda: PipelineWithFeatureNames(steps=[
    ('onehot', OneHotEncoder(sparse=False,dtype=int,categories = 'auto'))], feature_names = categorial_features)

categorical_pipeline_target = lambda: PipelineWithFeatureNames(steps=[
    ('numeric', TargetEncoder())], feature_names = categorial_features_target)

categorical_pipeline_ignore_unknown = lambda: PipelineWithFeatureNames(steps=[
    ('onehot', OneHotEncoder(sparse=False,dtype=int,categories = 'auto',handle_unknown = 'ignore'))], feature_names = categorial_features_ignore_unknown)

X_Transformer_col = lambda : DfColumnTransformer(transformers = [
    ('categorical_with_missing_values_const', categorical_with_missing_values_pipeline_const(),categorial_features_with_missing_values_const),
    ('categorical_with_missing_values_const_target', categorical_with_missing_values_pipeline_const_target(),categorial_features_with_missing_values_const_target),
    ('categorical_with_missing_values_impute', categorical_with_missing_values_pipeline_impute(),categorial_features_with_missing_values_impute),
    ('categorical_with_missing_values_impute_target', categorical_with_missing_values_pipeline_impute_target(),categorial_features_with_missing_values_impute_target),
    ('categorical',categorical_pipeline(),categorial_features),
    ('categorical_target',categorical_pipeline_target(),categorial_features_target),
    ('categorical_ignore_unknown',categorical_pipeline_ignore_unknown(),categorial_features_ignore_unknown),
    ('numeric_with_missing_values_impute',SimpleImputerWithFeatureNames(strategy='median',feature_names = numeric_columns_impute),numeric_columns_impute)])


cat_const_remove = []
cat_impute_remove = []

cat_remove = []

X_Transformer = lambda: Pipeline(steps = [('col_drop',ColumnDrop(['Utilities','Street','PoolQC'] + cat_remove + cat_impute_remove + cat_const_remove)),('col_transformer',X_Transformer_col()),('overfit_drop',OverfitDrop())])
X_Transformer_scaled = lambda : Pipeline(steps = [('col_drop',ColumnDrop(['Utilities','Street','PoolQC']  + cat_remove + cat_impute_remove + cat_const_remove)),('col_transformer',X_Transformer_col()),('overfit_drop',OverfitDrop()),('scaler',RobustScaler())])